In [1]:
import pandas as pd 
import numpy as np 

from sklearn.metrics.pairwise import cosine_similarity
from tqdm.auto import tqdm

c:\Users\minkyu\AppData\Local\anaconda3\envs\sd2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [50]:
text_value = "작업전 안전교육 실시와 안전관리자 안전점검 실시를 통한 재발 방지 대책 및 향후 조치 계획"

# 인덱스 0부터 963까지 총 964행을 가진 DataFrame 생성
sub1 = pd.DataFrame({"sub": [text_value] * 964})

In [41]:
sub2 = pd.read_csv("train2.csv")

In [42]:
sub2 = sub2['templeate']

In [44]:
sub3 = pd.read_csv("data/submissionV13.csv")
sub4 = pd.read_csv("data/submissionV18ckp3000.csv")

In [46]:
sub3 = sub3['재발방지대책 및 향후조치계획']
sub4 = sub4['재발방지대책 및 향후조치계획']

In [51]:
df = pd.concat([sub1, sub2, sub3, sub4], axis=1)


In [52]:
df.columns = ['sub1', 'sub2', 'sub3', 'sub4']


In [53]:
df.columns

Index(['sub1', 'sub2', 'sub3', 'sub4'], dtype='object')

In [58]:
df = df[:964]

In [4]:
from sentence_transformers import SentenceTransformer

# Embedding Vector 추출에 활용할 모델(jhgan/ko-sbert-sts) 불러오기
model = SentenceTransformer('jhgan/ko-sbert-sts', use_auth_token=False)

c:\Users\minkyu\AppData\Local\anaconda3\envs\sd2\lib\site-packages\sentence_transformers\SentenceTransformer.py:195: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v4 of SentenceTransformers.
  warnings.warn(


In [60]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

# 예시: df는 sub1, sub2, sub3, sub4 컬럼을 포함하는 데이터프레임
# model은 텍스트를 벡터로 인코딩하는 모델입니다.

# 각 행별 대표 텍스트와 유사도 정보를 저장할 리스트 초기화
representative_texts = []  # 각 행별 대표 텍스트 저장
cosine_scores = []         # 각 행별 대표 텍스트와 다른 텍스트 간 유사도 기록

# 모델을 통해 텍스트를 벡터로 인코딩하는 예시 함수가 있다고 가정합니다.
# 예: model.encode(text)
# model은 이미 초기화된 인코딩 모델이어야 합니다.

# 각 행에 대해 sub1~sub5 컬럼의 텍스트를 비교
for idx, row in tqdm(df.iterrows(), total=len(df)):
    # sub1부터 sub5까지의 텍스트를 리스트로 추출합니다.
    texts = [row[f"sub{i}"] for i in range(1, 5)]
    
    # 각 텍스트를 벡터화합니다.
    vectors = np.stack([model.encode(text) for text in texts])
    
    # 텍스트들 간의 코사인 유사도 행렬 계산
    similarity = cosine_similarity(vectors, vectors)
    
    # 각 텍스트에 대해 다른 텍스트들과의 평균 유사도를 계산합니다.
    avg_similarity = similarity.mean(axis=1)
    
    # 평균 유사도가 가장 높은 텍스트의 인덱스를 선택합니다.
    rep_idx = avg_similarity.argmax()
    
    # 대표 텍스트와 해당 텍스트의 유사도 벡터를 저장합니다.
    representative_texts.append(texts[rep_idx])
    cosine_scores.append(similarity[rep_idx].tolist())

# 결과를 DataFrame의 새로운 컬럼에 추가합니다.
df["대표재발방지대책"] = representative_texts
df["대표유사도"] = cosine_scores

# 결과 출력
print(df.head())

100%|██████████| 964/964 [00:42<00:00, 22.90it/s]

                                                sub1  \
0  작업전 안전교육 실시와 안전관리자 안전점검 실시를 통한 재발 방지 대책 및 향후 조...   
1  작업전 안전교육 실시와 안전관리자 안전점검 실시를 통한 재발 방지 대책 및 향후 조...   
2  작업전 안전교육 실시와 안전관리자 안전점검 실시를 통한 재발 방지 대책 및 향후 조...   
3  작업전 안전교육 실시와 안전관리자 안전점검 실시를 통한 재발 방지 대책 및 향후 조...   
4  작업전 안전교육 실시와 안전관리자 안전점검 실시를 통한 재발 방지 대책 및 향후 조...   

                                                sub2  \
0  작업자 안전교육 실시와 작업 위험구간 안전조치 철저, 안전점검 및 안전조치 여부 전...   
1  작업전 작업자 안전교육 강화, 안전수칙 준수 특별안전교육 실시, 작업지휘자 관리감독...   
2  이동통로 확보 관리와 작업 전 안전교육 철저 및 정기적 근로자 안전교육 시행을 통한...   
3  작업전 안전교육 및 특별안전교육 실시와 안전관리 및 안전교육 철저를 통한 재발 방지...   
4  이동통로 확보 관리와 작업 전 안전교육 철저 및 정기적 근로자 안전교육 시행을 통한...   

                                                sub3  \
0  타설 작업 시 펌프카 아웃트리거 바닥 고임목을 3단 이상으로 보강하고, 작업 위치를...   
1                      작업자 안전교육 강화와 작업 전 안전수칙 준수 철저.   
2           작업자 안전교육 강화와 작업발판 설치 확대를 통한 재발 방지 대책 수립.   
3           작업 발판 설치 확대와 정기적인 안전교육을 통해 사고 예방을 강화합니다.   
4                      작업자 안전교육 강화와 작업 전 안전점검 


C:\Users\minkyu\AppData\Local\Temp\ipykernel_18628\4108430798.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["대표재발방지대책"] = representative_texts
C:\Users\minkyu\AppData\Local\Temp\ipykernel_18628\4108430798.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["대표유사도"] = cosine_scores


In [62]:
df['대표재발방지대책']

0      작업자 안전교육 실시와 작업 위험구간 안전조치 철저, 안전점검 및 안전조치 여부 전...
1      작업전 작업자 안전교육 강화, 안전수칙 준수 특별안전교육 실시, 작업지휘자 관리감독...
2      작업전 안전교육 실시와 안전관리자 안전점검 실시를 통한 재발 방지 대책 및 향후 조...
3      작업전 안전교육 및 특별안전교육 실시와 안전관리 및 안전교육 철저를 통한 재발 방지...
4             작업자 안전교육 강화와 작업 중 안전수칙 준수를 위한 현장 감독 강화 실시.
                             ...                        
959    작업전 안전교육 실시와 안전관리자 안전점검 실시를 통한 재발 방지 대책 및 향후 조...
960    작업전 안전교육 철저와 작업중 위험요인 파악 및 제거를 통한 재발 방지 대책 및 향...
961    안전교육 실시와 현장 내 작업지시사항 철저 이행, 안전관리 교육 이행, 안전위험요소...
962    작업전 작업자 안전교육 강화, 안전수칙 준수 특별안전교육 실시, 작업지휘자 관리감독...
963    작업전 안전교육 실시와 안전관리자 안전점검 실시를 통한 재발 방지 대책 및 향후 조...
Name: 대표재발방지대책, Length: 964, dtype: object

In [63]:


# 문장 리스트를 입력하여 임베딩 생성
pred_embeddings = model.encode(df['대표재발방지대책'])

In [64]:
submission = pd.read_csv("sample_submission.csv", encoding = 'utf-8-sig')
# 최종 결과 저장
submission.iloc[:,1] = df['대표재발방지대책']
submission.iloc[:,2:] = pred_embeddings
submission.head()
# 최종 결과를 CSV로 저장
submission.to_csv('신기술V2.csv', index=False, encoding='utf-8-sig')